<a href="https://colab.research.google.com/github/SanghunOh/study_machinglearning/blob/main/codes/sanghunoh/reports/Term_2_Finetuning_MPIE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Connect Drive

In [ ]:
path_root = '/content/drive/MyDrive/datas/'

# 작업 경로 설정
import os
os.chdir(path_root)
!pwd

/content/drive/.shortcut-targets-by-id/1tHYgEz3lQF-LmtFPXRgMBDh4UN54qeAs/datas


In [ ]:
# !unzip -o ./mpie_30_shuffle.zip -d ./mpie_30_shuffle

In [ ]:
!ls

dataCh4_7.mat		iris_shuffle.mat	 mpie_30_shuffle
dataCh4_7.zip		iris.zip		 mpie_30_shuffle.zip
digitimages_binary.zip	matlab_iris_shuffle.mat  number_images


###Load Dataset

In [ ]:
path = path_root + 'mpie_30_shuffle/'

def loadDatasetFromCSV(_datafilename, _labelfilename, label_column):
  _feature_csv = pd.read_csv(path + _datafilename, dtype=np.float32, header=None) # image features of train data
  _feature_flatten = _feature_csv.values.flatten()
  _feature_reshape = np.reshape(_feature_flatten, (_feature_csv.shape[0], 32,-1))

  _label_csv = pd.read_csv(path + _labelfilename, dtype=np.float32, header=None) # labels of train data
  # print(f'_label_csv : {_label_csv.shape}')
  _label = _label_csv[label_column] # get label you want

  return _feature_reshape, _label

In [ ]:
import pandas as pd
import numpy as np

# train
train_feature_reshape, train_label = loadDatasetFromCSV('Traindata.csv', 'Trainlabel.csv', 0)
# test
validation_feature_reshape, validation_label = loadDatasetFromCSV('Testdata.csv', 'Testlabel.csv', 0)

train_feature_reshape.shape, train_label.shape, validation_feature_reshape.shape, validation_label.shape

((18777, 32, 32), (18777,), (5086, 32, 32), (5086,))

In [ ]:
np.unique(train_label)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29.], dtype=float32)

In [ ]:
import tensorflow as tf

def preprocessDataset(features_reshape, labels, batch_size=64, shuffle_buffer_size=100):
  _batch_size = batch_size
  _shuffle_buffer_size = shuffle_buffer_size

  _features = features_reshape
  _labels = labels
  _dataset_tensors = tf.data.Dataset.from_tensor_slices((_features, _labels))

  if _shuffle_buffer_size == None:
    _dataset_tensors = _dataset_tensors.batch(_batch_size)
  else :
    _dataset_tensors = _dataset_tensors.shuffle(_shuffle_buffer_size).batch(_batch_size)

  return _dataset_tensors

In [ ]:
train_dataset = preprocessDataset(train_feature_reshape, train_label)
train_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 32, 32), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [ ]:
# train_dataset?

In [ ]:
validation_dataset = preprocessDataset(validation_feature_reshape, validation_label, shuffle_buffer_size=None)
validation_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 32, 32), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

###callback function for fit time

In [ ]:
import time

class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
TimeHistory()

### fit model

In [ ]:
_epochs = 10
_batch_size = int(len(list(train_dataset)) / (len(list(train_dataset))/2))   # Just Check Model params quickly
# _batch_size = len(list(train_dataset))

def model_fit(_param, _train_dataset, _validation_dataset):
  tf.keras.backend.clear_session()

  model = make_model(**_param)
  time_callback = TimeHistory()
  history = model.fit(_train_dataset, epochs=_param['epochs'], validation_data=_validation_dataset, callbacks=[time_callback], steps_per_epoch=_batch_size)
  execution_time = sum(time_callback.times)
  return model, history, execution_time, _param

In [ ]:
label_class = 0
# train
train_feature_reshape, train_label = loadDatasetFromCSV('Traindata.csv', 'Trainlabel.csv', label_class)
class_cnt = len(np.unique(train_label))
# test
validation_feature_reshape, validation_label = loadDatasetFromCSV('Testdata.csv', 'Testlabel.csv', label_class)

train_dataset = preprocessDataset(train_feature_reshape, train_label)
validation_dataset = preprocessDataset(validation_feature_reshape, validation_label, shuffle_buffer_size=None)

param = {'_class_cnt':class_cnt, 'epochs':2, 'label_class':label_class, '_kernel_initializer':None
         , '_activation':'sigmoid', 'batch_normal':False, '_optimizer':'RMSprop', 'filters':16 , '_padding':'valid'
         , 'pool_type':'max', '_kernel_size': (3,3), '_strides': (1,1), '_pool_size': (2,2)}

model_fit(param, train_dataset, validation_dataset)

Epoch 1/2
2/2 [==============================] - 4s 2s/step - loss: 3.9965 - accuracy: 0.0234 - val_loss: 3.6237 - val_accuracy: 0.0203
Epoch 2/2
2/2 [==============================] - 2s 2s/step - loss: 3.8177 - accuracy: 0.0312 - val_loss: 3.5895 - val_accuracy: 0.0203


(<keras.engine.sequential.Sequential at 0x7f8bec93db50>,
 6.299786567687988,
 {'_activation': 'sigmoid',
  '_class_cnt': 30,
  '_kernel_initializer': None,
  '_kernel_size': (3, 3),
  '_optimizer': 'RMSprop',
  '_padding': 'valid',
  '_pool_size': (2, 2),
  '_strides': (1, 1),
  'batch_normal': False,
  'epochs': 2,
  'filters': 16,
  'label_class': 0,
  'pool_type': 'max'})

### fit with multi params
filters : larger than 16

In [ ]:
param = {'_class_cnt':class_cnt, 'epochs':2, 'label_class':label_class, '_kernel_initializer':None
         , '_activation':'sigmoid', 'batch_normal':False, '_optimizer':'Nadam', 'filters':16 , '_padding':'same'
         , 'pool_type':'average', '_kernel_size': (3,3), '_strides': (1,1), '_pool_size': (2,2)}


In [ ]:
# param 6
params = list()
param_key_list = list(param.keys())
epochs_list = [10, 50, 100]
_kernel_initializer_list = [None, 'glorot_uniform', 'he_normal']
_activation_list = ['sigmoid', 'relu']
_optimizer_list= ['Adam', 'Nadam']
batch_normal_list = [False, True]
filters_list = [16, 32]
_padding_list = ['same', 'valid']
_kernel_size_list = [(3,3), (5,5)]
_strides_list = [(1,1), (3,3)]
pool_type_list = ['average', 'max']

for filters in filters_list:
  for _padding in _padding_list:
    for _kernel_initializer in _kernel_initializer_list:
      for _activation in _activation_list:
        for _optimizer in _optimizer_list:
          for batch_normal in batch_normal_list:
            for epochs in epochs_list:
              for _kernel_size in _kernel_size_list:
                for _strides in _strides_list:
                  for pool_type in pool_type_list:
                    param_dict = dict()
                    param_dict['filters'] = filters
                    param_dict['_padding'] = _padding
                    param_dict['_kernel_initializer'] = _kernel_initializer
                    param_dict['_activation'] = _activation
                    param_dict['_optimizer'] = _optimizer
                    param_dict['batch_normal'] = batch_normal
                    param_dict['epochs'] = epochs
                    param_dict['_kernel_size'] = _kernel_size
                    param_dict['_strides'] = _strides
                    param_dict['pool_type'] = pool_type
                    # print(param_dict)
                    params.append(param_dict)
len(params)

2304

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%Y%m%d%H%M")
print("date and time =", dt_string)

import pickle

now = 2022-05-28 11:59:27.224467
date and time = 202228051159


In [ ]:
# params = params[5:7]
label_classes = [1, 2, 4]
results_list = list()
for label_class in label_classes:
  train_feature_reshape, train_label = loadDatasetFromCSV('Traindata.csv', 'Trainlabel.csv', label_class)
  class_cnt = len(np.unique(train_label))
  # test
  validation_feature_reshape, validation_label = loadDatasetFromCSV('Testdata.csv', 'Testlabel.csv', label_class)

  train_dataset = preprocessDataset(train_feature_reshape, train_label)
  validation_dataset = preprocessDataset(validation_feature_reshape, validation_label, shuffle_buffer_size=None)

  results = list()          
  for idx, param in enumerate(params):
    param['_class_cnt'] = class_cnt
    param['label_class'] = label_class
    print('-'*5 +'[ '+ str(idx) + ' ]'+ '-'*5 + str(param))
    results.append(model_fit(param, train_dataset, validation_dataset))
  # save
  with open(f'results_CNN_label_{label_class}_{dt_string}.pickle', 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)
  results_list = results_list + results


In [ ]:
# save
with open(f'results_CNN_{dt_string}.pickle', 'wb') as f:
    pickle.dump(results_list, f, pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: ram://9747bf27-8aa7-4168-b9dc-7b00a406f415/assets
INFO:tensorflow:Assets written to: ram://ab1328e4-c958-43b9-a909-85e328d58d25/assets
INFO:tensorflow:Assets written to: ram://2024e3d0-87ad-469f-8ee8-ec26453132a7/assets
INFO:tensorflow:Assets written to: ram://0e34876c-925a-47c2-8bc9-936d5fd52102/assets
INFO:tensorflow:Assets written to: ram://4296993a-7b7f-400f-9ad7-7f2d856b5b81/assets
INFO:tensorflow:Assets written to: ram://254d7a40-6e1e-4a21-868f-b00c6205a414/assets
